## Methanol-Synthese

T=493;15K

p=50 bar

### Bilanzen
Stoffbilanzen

$\dot n_i = \dot n_{i, 0} + \sum_{j}{\nu_ij \xi_j}$ 

Energiebilanz

$0 = \dot Q + \sum\limits_{j}{\xi_j \Delta H_j}$

### Gleichgewichtskonstanten

$\begin{array}{ll}
exp \left(- \frac{\Delta G_i}{R T} \right) &= K_p K_{\phi^{eq}} = K_x \prod\limits_{i} \left( \frac{p}{p^0}\right)^{\nu_i}  K_{\phi^{eq}} \\
&=\prod\limits_{i} (n_i)^{\nu_i}\left( \frac{p}{p^0}\right)^{\sum\limits_{i} \nu_i}(n)^{-\sum\limits_{i} \nu_i}   K_{\phi^{eq}}\end{array}$

$p^° = 1 bar$

Idealer Gas, $K_{\phi^{eq}}=1$ 

**Methode A)** Geringe Veränderung der Reaktionsenthalpie mit der Temperatur

Van't Hoff, $\frac{d ln K}{dT} = -\frac{\Delta H}{R T^2} \sim \Rightarrow ln \left(\frac{K}{K'} \right) = -\frac{\Delta H^0}{R}\left(\frac{1}{T} - \frac{1}{T'} \right)$

$\begin{array}{ll}
K_{(493,15K)} &= K_{(298,15K)} \times exp \left[-\frac{\Delta H^0}{R}\left(\frac{1}{493,15 K} - \frac{1}{298,15 K} \right)\right] \\
&= \prod_i (n_i)^{\nu_i}\left( \frac{p}{p^0}\right)^{\sum_i \nu_i}(n)^{-\sum_i \nu_i}\end{array}$ 

**Methode B)** Wechselwirkung der Reaktionsenthalpie mit der Temperatur [SVNA]

$\Delta H^° = \Delta H_0^° + R \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}dT}$

$\Delta S^° = \Delta S_0^° + R \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}\frac{dT}{T}}$

$\Delta G^° = \Delta H^° - T \Delta S^° = \Delta H_0^° + R \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}dT} - T \Delta S_0^° - R T \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}\frac{dT}{T}}$

$\Delta S_0^° = \frac{\Delta H_0^° - \Delta G_0^°}{T_0}$

$\Delta G^° = \Delta H_0^° - \frac{T}{T_0}(\Delta H_0^° -\Delta G_0^°) + R \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}dT} - R T \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}\frac{dT}{T}}$

$\begin{array}{ll}
K_{(T)} &= exp \left(-\frac{\Delta H_0^°}{R T} + \frac{(\Delta H_0^° -\Delta G_0^°)}{R T_0} - \frac{1}{T}\int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}dT} + \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}\frac{dT}{T}}\right) \\
&= \prod_i (n_i)^{\nu_i}\left( \frac{p}{p^0}\right)^{\sum_i \nu_i}(n)^{-\sum_i \nu_i}\end{array}$ 

Somit läßt sich K(T) bestimmen, insofern man über einen Ausdruck für $Cp_i(T)$ verfügt. Bei geringer Veränderung der Wärmekapazität Cp im Temperatur-Bereich kann man auch einen bestimmten Mittelwert als ~konstant einsetzen.

**Methode C)** Gibbssche Energie-Funktion - Gef(T) - aus Thermochemischen Tabellen [BP]

$-Gef(T) = -[G(T)-H(298,15)]/T$

$-R ln(K) = \sum\nu_i Gef_i - \sum \nu_i H_i(298,15K)/T$

In thermochemischen Tabellen [BP] sind die Werte -Gef(T) verfügbar. 

### Literaturhinweise
* [SVNA] Smith J.M., Van Ness H.C., Abbott M.M.; Introduction to chemical engineering thermodynamics; 6th ed.; McGraw-Hill; New York; 2001; S. 458-462.
* [BP] Barin Isan, Platzki Gregor; Thermochemical data of pure substances; 3. ed.; VCH; New York; 1995.

In [64]:
from scipy import optimize
import numpy as np

p = 50. # bar
temp = 273.15 + 220. # K

namen = ['CO', 'H2', 'CO2', 'H2O', 'CH3OH']

n0co = 50.
n0h2 = 100.
n0co2 = 10.
n0h2o = 0.
n0ch3oh = 0.

ne = np.array([n0co, n0h2, n0co2, n0h2o, n0ch3oh])

nuij = np.array([[-1, -2, 0, 0, +1] ,
                 [0, -3, -1, +1, +1], 
                 [-1, +1, +1, -1, 0]]).T

delta_h_298 = np.array(
    [-110.54, 0, -393.50, -241.82,-201.16]) * 1000 # J/mol

delta_g_298 = np.array(
    [-137.18, 0, -394.36, -228.62, -162.44]) * 1000 # J/mol

cp_298 = np.array(
    [29.14, 28.83, 37.13, 33.59, 43.81] # J/(mol K)
)

cp_493 = np.array(
    [29.76, 29.25, 44.39, 35.16, 58.95] #  J/(mol K)
)

delta_hr_298 = nuij.T.dot(delta_h_298)

delta_gr_298 = nuij.T.dot(delta_g_298)

delta_cp_298_493 = np.mean(
    np.array([
        nuij.T.dot(cp_298), nuij.T.dot(cp_493)
    ]), axis=0)

k_298 = np.exp(-delta_gr_298/(8.314*298.15))
k_493 = np.exp(-(
    delta_hr_298 - 493.15/298.15*(
        delta_hr_298 - delta_gr_298
    ) + delta_cp_298_493 * (
        493.15 - 298.15
    ) - delta_cp_298_493 * 493.15 *  np.log(493.15/298.15)
)/(8.314 * 493.15))

for i, f in enumerate(delta_hr_298):
    print('Delta H_' + str(i+1) + '(298.15K)=' + str(f/1000.) + 'kJ/mol')

print('\n')
for i, f in enumerate(k_493):
    print('K' + str(i+1) + '(493K)=' + str(f))
print('\n')
    
def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nch3oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nco + n0co - xi1
    f2 = -nh2  + n0h2 -2*xi1
    f3 = -nch3oh + n0ch3oh +xi1
    f4 = -k_493[0] * (nco * nh2**2) + \
            nch3oh * (p/1.)**-2 * (nco + nh2 + nch3oh)**-(-2)
    
    return [f1, f2, f3, f4]

n0 = np.array([n0co, n0h2, n0ch3oh])
x0 = np.append(n0, [0.])
#n0 = np.array([21.125, 42.25, 28.875])
#x0 = np.append(n0, [28.875])
sol = optimize.root(fun, x0)
f_final = - sol.x[:3].reshape([3,1]) + ne[[0,1,4]].reshape([3,1]) + nuij[:,0][[0,1,4]].reshape([3,1])*sol.x[-1]

print(sol)
print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

print('\n\n')
print('T=493.15K, p=50 bar')
print('Lösung für nur einzige Reaktion (ohne CO2):\n')
for i, f in enumerate(sol.x[:2]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
print('n_' + namen[-1] + '= ' + str(sol.x[2]) + ' mol')

# Lösung des einfacheren Falls in schwierigerem Fall einwenden.
def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nco2 = x_vec[2]
    nh2o = x_vec[3]
    nch3oh = x_vec[4]
    xi1 = x_vec[5]
    xi2 = x_vec[6]
    xi3 = x_vec[7]
    
    f1 = -nco + n0co - xi1 +0 -xi3
    f2 = -nh2  + n0h2 -2*xi1 -3*xi2 +xi3
    f3 = -nco2 + n0co2 +0 -xi2 +xi3
    f4 = -nh2o + n0h2o +0 +xi2 -xi3
    f5 = -nch3oh + n0ch3oh +xi1 +xi2 -0
    f6 = -k_493[0] * (nco * nh2**2) + \
        nch3oh * (p/1.)**-2 * (nco + nh2 + nco2 + nh2o + nch3oh)**-(-2)
    f7 = -k_493[1] * (nco2 * nh2**3) + \
        nch3oh * nh2o * (p/1.)**-2 * (nco + nh2 + nco2 + nh2o + nch3oh)**-(-2)
    f8 = -k_493[2] * (nco * nh2o) + \
        nco2 * nh2 * (p/1.)**0 * (nco + nh2 + nco2 + nh2o + nch3oh)**-0
    
    return [f1, f2, f3, f4, f5, f6, f7, f8]

n0 = np.array([sol.x[0], sol.x[1], n0co2, 0, sol.x[2]])
x0 = np.append(n0, [sol.x[3], 0, 0])
#n0 = np.array([1.79111141, 3.58222282, 10., 0, 48.20888859])
#x0 = np.append(n0, [48.20888859, 0, 0])
sol = optimize.root(fun, x0)

f_final = - sol.x[:5].reshape([5,1]) + ne.reshape([5,1]) + nuij.dot(sol.x[5:].reshape([3,1]))

print('\n\n')
print('T=493.15K, p=50 bar')
print('Lösung für alle drei Reaktionen, mit CO2:\n')
for i, f in enumerate(sol.x[:5]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')

print('\n')

for i, f in enumerate(sol.x[5:]):
    print('xi_' + str(i) + '= ' + str(f) + ' mol')

print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

Delta H_1(298.15K)=-90.62kJ/mol
Delta H_2(298.15K)=-49.48kJ/mol
Delta H_3(298.15K)=-41.14kJ/mol


K1(493K)=0.00879175269094
K2(493K)=5.71150952797e-05
K3(493K)=153.930456526


    fjac: array([[ -2.70866601e-02,  -2.35823418e-14,  -2.45190657e-18,
         -9.99633089e-01],
       [ -3.56821545e-01,  -9.34122531e-01,  -2.01414847e-16,
          9.66865143e-03],
       [ -1.95429987e-01,   7.47062916e-02,  -9.77853797e-01,
          5.29548861e-03],
       [  9.13100114e-01,  -3.49047372e-01,  -2.09289159e-01,
         -2.47419105e-02]])
     fun: array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.42108547e-14])
 message: 'The solution converged.'
    nfev: 16
     qtf: array([  1.08690481e-08,  -1.05127610e-10,  -5.75780467e-11,
         2.69019723e-10])
       r: array([ 36.91854208, -14.10235893,  -8.45578877,   0.06666537,
         1.07052337,   0.08178608,   2.22468379,   1.02264777,
        -0.93204606,  -0.42331492])
  status: 1
 success: True
       x: array

In [65]:
for part in sol.x:
    print str(part).replace('.',',')

14,8812942217
29,3826513485
9,87335430171
0,126645698294
35,2453514766
1556,16517416
-1520,91982269
-1521,04646838


In [2]:
# Vorsicht! K-Wert im Aspen Plus
k1_493 = 0.00267615835

def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nch3oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nco + n0co - xi1
    f2 = -nh2  + n0h2 -2*xi1
    f3 = -nch3oh + n0ch3oh +xi1
    f4 = -k1_493 * (nco * nh2**2) + nch3oh * (p/1.)**-2 * (nco + nh2 + nch3oh)**-(-2)
    
    return [f1, f2, f3, f4]
print fun([21.125, 42.25, 28.875, 28.875 ])
print fun([11.4243863316, 22.8487726633, 38.5756136684, 38.5756136684])

[42.25/21.125, 22.8487726633/11.4243863316]

[0.0, 0.0, 0.0, -2.6252250719152244]
[0.0, -1.000017846308765e-10, 0.0, 65.9260634809681]


[2.0, 2.0000000000087534]

# SVN 13.6 (S. 485)
$C_2H_4(g) + H_2 O(g) \rightleftharpoons C_2H_5OH(g)$

In [3]:
p = 35. # bar
temp = 523.15 # K

namen = ['C2H4', 'H2O', 'C2H5OH']

n0c2h4 = 2.
n0h2o = 5.
n0c2h5oh = 0.

ne = np.array([n0c2h4, n0h2o, n0c2h5oh])

nuij = np.array([[-1, -1, +1]]).T

delta_h_1_298 = (-(+52.467) -(-241.826) + -234.906)*1000

delta_g_1_298 = (-(+68.420) -(-228.620) + -168.200)*1000

k1_298 = np.exp(-delta_g_1_298/(8.314*298.15))

# Angenommen, dass Delta_H fast im Interval zwischen 298.15K und 443.15K 
# unabhängig von der Temperatur sei...
k1_523 = k1_298 * np.exp(-delta_h_1_298/8.314*( 1/523.15 - 1/298.15 ))

print('\n')
for i, f in enumerate([k1_523]):
    print('K' + str(i+1) + '(523.15K)=' + str(f))
print('\n')


def fun(x_vec):    
    nc2h4 = x_vec[0]
    nh2o = x_vec[1]
    nc2h5oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nc2h4 + n0c2h4 -xi1
    f2 = -nh2o  + n0h2o -xi1
    f3 = -nc2h5oh + n0c2h5oh +xi1
    f4 = -k1_523 * (nc2h4 * nh2o) + \
            nc2h5oh * (p/1.)**-1. * (nc2h4 + nh2o + nc2h5oh)**(-(-1))
    
    return [f1, f2, f3, f4]

n0 = np.array([(0.233), n0h2o, n0c2h5oh])
x0 = np.append(n0, [0.233])
sol = optimize.root(fun, x0)
f_final = - sol.x[:3].reshape([3,1]) + n0.reshape([3,1]) + nuij.reshape([3,1])*sol.x[-1]

print(sol)
print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

print('\n\n')
print('T=' + str(temp) +'K, p=' + str(p) +'bar')
for i, f in enumerate(sol.x[:-1]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
print('x1= ' + str(sol.x[-1]))



K1(523.15K)=0.00932292186355


    fjac: array([[ -9.99223948e-01,   2.10438579e-10,  -1.04114753e-10,
         -3.93891191e-02],
       [  5.01145052e-04,  -9.99919060e-01,  -8.25469011e-10,
         -1.27130629e-02],
       [ -6.22922226e-03,  -2.01224058e-03,  -9.87413727e-01,
          1.58023035e-01],
       [  3.88902089e-02,   1.25628023e-02,  -1.58158566e-01,
         -9.86567583e-01]])
     fun: array([  5.55111512e-17,  -1.66533454e-16,   0.00000000e+00,
         0.00000000e+00])
 message: 'The solution converged.'
    nfev: 11
     qtf: array([  2.24060002e-13,   7.19101664e-14,  -8.98671873e-13,
         5.61057783e-12])
       r: array([  1.00077665e+00,   5.01569423e-04,  -6.31454054e-03,
         9.98252337e-01,   1.00008095e+00,  -2.03805403e-03,
         9.99104324e-01,   1.01274671e+00,  -9.75274364e-01,
        -2.33946887e-01])
  status: 1
 success: True
       x: array([ 1.62885912,  4.62885912,  0.37114088,  0.37114088])



Zustand der Optimisierungs-Funktionen


# Min(G)

$\Delta G_{f i}^{°} + R T ln(y_i \hat{\phi_i} P/P^°) + \sum\limits_{k}{ \lambda_k a_{ik}}=0 \hspace{10mm} (i=1,2,...,N)$

$\sum\limits_{i}{n_i a_{ik}}=A_k \hspace{10mm} (k = 1,2,...,\omega)$